# Evoluční modelování

## importy

In [35]:
import numpy as np
import numpy.typing as npt
import pandas as pd
import plotly.graph_objects as go
from typing import Callable, Any, Self
from time import perf_counter
from statistics import mean, stdev
import scipy
from dataclasses import dataclass
from collections import deque

## pomocný kód

### testovací funkce

Testovací ptimaliyační funkce. Nejsou specifikovány samostatně, ale jsou přímo napsané pro 2D.

In [36]:
def ackley(x: npt.NDArray, a=20, b=0.2, c=2 * np.pi):
    d = len(x)  # dimension of input vector x
    sum_sq_term = -a * np.exp(-b * np.sqrt(sum(x * x) / d))
    cos_term = -np.exp(sum(np.cos(c * x) / d))
    return a + np.exp(1) + sum_sq_term + cos_term


def himmel_baum(x: npt.NDArray):
    return ((x[0] ** 2 + x[1] - 11) ** 2) + (((x[0] + x[1] ** 2 - 7) ** 2))

### měřící funkce

Měřící funkce která reportuje statistiku po n specifikovaných opakování. Zároveň vrací výsledek posledního opakování.

In [74]:
def measure(
    method: Callable[..., tuple[float, int, int, pd.DataFrame]],
    trials: int = 10,
    **kwargs,
) -> pd.DataFrame:
    if trials < 1:
        raise Exception("need atleast specify 1 trial")

    res: tuple[float, int, int, pd.DataFrame] = (0, 1, 1, pd.DataFrame())
    times = []
    results = []
    iterations = []
    evaluations = []

    for _ in range(trials):
        start = perf_counter()
        res = method(**kwargs)
        times.append(perf_counter() - start)
        results.append(res[0])
        iterations.append(res[1])
        evaluations.append(res[2])

    print(
        f"avg. time: {mean(times)} +-{stdev(times)} seconds, result: {mean(results)} +-{stdev(results)}, iterations: {mean(iterations)} +-{stdev(iterations)}, evaluations: {mean(evaluations)} +-{stdev(evaluations)}"
    )
    return res[-1]

### animace 

Pomocná funkce pro vytvoření animace pro 3d za použití knihovny Plotly.

In [38]:
def frame_args(duration: int) -> dict[str, Any]:
    """
    Create dictinary with setting's for frame

    Args:
        duration (int): Duration of the frame in ms.

    Returns:
        dict[str,Any]: Dictionary settings for plotly frame animation,
    """
    return {
        "frame": {"duration": duration},
        "mode": "immediate",
        "fromcurrent": True,
        "transition": {"duration": duration, "easing": "cubic"},
    }


def animate(
    df: pd.DataFrame,
    function: Callable = scipy.optimize.rosen,
    x_min: float = -1,
    x_max: float = 1,
    y_min: float = -1,
    y_max: float = 1,
    mesh_points_x: int = 100,
    mesh_points_y: int = 100,
    frame_duration: int = 100,
    marker_size: float = 4.0,
) -> go.Figure:
    """
    Creating 3D animation plot in plotly

    Args:
        df (pd.DataFrame): dataframe with given columns:
            x_coord:  x coordination of mark
            y_coord: y coordination of mark
            value: function value (z value in the plot)
            iteration: Iteration of Algorithm. The marks are grouped into individual frames by iterations.
        function (Callable, optional): Function which plot as landscape. Defaults to scipy.optimize.rosen.
        x_min (float, optional): Minimal value of landscape function on x axis. Defaults to -1.
        x_max (float, optional): Maximal value of landscape function on x axis. Defaults to 1.
        y_min (float, optional): Minimal value of landscape function on y axis. Defaults to -1.
        y_max (float, optional):  Maximal value of landscape function on y axis. Defaults to 1.
        mesh_points_x (int, optional): How many point's of landscape function to evaluate along x axis. Defaults to 100.
        mesh_points_y (int, optional): How many point's of landscape function to evaluate along y axis Defaults to 100.
        frame_duration (int, optional): Frames duration in ms  . Defaults to 100.
        marker_size (float, optional): Size of markers (specified points) on surface. Defaults to 4..

    Returns:
        go.Figure: Figure with 3D animation.
    """
    fig = go.Figure(
        frames=[
            go.Frame(
                data=go.Scatter3d(
                    x=data["x_coord"],
                    y=data["y_coord"],
                    z=data["value"],
                    mode="markers",
                    marker={"size": marker_size},
                ),
                name=str(
                    name
                ),  # you need to name the frame for the animation to behave properly
            )
            for name, data in df.groupby("iteration")
        ]
    )

    # Add data to be displayed before animation starts (1 beggining iteration)
    filtered = df[df.iteration == 0]
    fig.add_scatter3d(
        x=filtered["x_coord"],
        y=filtered["y_coord"],
        z=filtered["value"],
        mode="markers",
        marker={"size": marker_size},
        showlegend=False,
    )
    x = np.linspace(x_min, x_max, mesh_points_x)
    y = np.linspace(y_min, y_max, mesh_points_y)
    x, y = np.meshgrid(x, y)
    fig.add_surface(
        x=x,
        y=y,
        z=function(np.asarray((x, y))),
        contours={"z": {"show": True}},
        showlegend=False,
        showscale=False,
    )

    # define animation slider
    sliders = [
        {
            "pad": {"b": 10, "t": 60},
            "len": 0.9,
            "x": 0.1,
            "y": 0,
            "steps": [
                {
                    "args": [[frame.name], frame_args(frame_duration)],  # type: ignore
                    "label": str(index),
                    "method": "animate",
                }
                for index, frame in enumerate(fig.frames)
            ],
        }
    ]

    # Layout of the figure
    fig.update_layout(
        updatemenus=[
            {
                "buttons": [
                    {
                        "args": [None, frame_args(frame_duration)],
                        "label": "&#9654;",  # play symbol
                        "method": "animate",
                    },
                    {
                        "args": [[None], frame_args(frame_duration)],
                        "label": "&#9724;",  # pause symbol
                        "method": "animate",
                    },
                ],
                "direction": "left",
                "pad": {"r": 10, "t": 70},
                "type": "buttons",
                "x": 0.1,
                "y": 0,
            }
        ],
        sliders=sliders,
    )

    return fig

## simulované žíhání

Algoritmus simulovaného žíhání má původ ve fyzice, přesněji v metalurgii (odtuď i název žíhání). Hlavní princip spočívá v náhodném prozkoumávání prostoru kolem předem stanoveného bodu.
Pokud je bod lepší, automaticky ho příjmeme. V opačném případě za pomoci speciálního kritéria spočítáme pravděbodobnost, jestli daný bod příjmeme.
Díky tomuto ihned automaticky nezavrhujeme zhoršení a nemusí dojít k uváznutí  lokálním minimu.Toto kritérium se nazívá metropolisovo kritérium a je definován následovně.

$$ Metropolis(\Delta y) = \left\{
 \begin{array}{ll}
      1 & \Delta y \le 0 \\
      e^{-\frac{\Delta y}{t}} & \Delta y > 0 \\
\end{array} 
\right. $$

Jak si lze všimnout. Toto kritérium můžeme ovlivňovat parametrem t(teplotou), odtuď je inspirace z metalurgie, kdy postupním  ochlazováním snižujeme pravděbodobnost přijímání horšího bodu.

Možným vylepšením je lepší prohledávání prostoru na základě poměru. K tomuto se dodefinovává následující funkce pro výpočet proměnlivé délky kroku:

$$ v_i= \left\{
      \begin{array}{ll}
      v_i (1 + c_i \frac{a_i - 0.6}{0.4}) & a_i > 0.6 \\
      v_i (1 + c_i\frac{0.4-a_i}{0.4})^{-1} & a_i < 0.4 \\ 
      v_i
      \end{array}

\right.
\\
a_i \subset <0, 1> 
 $$ 

 Pokud platí a_i > 0.6 (poměr přiatých bodů vůči zamítnutým je větší než 0.6), tak si krok v dané dimenzi zvětšíme. Naopak pokud je úspěšnost menší než 0.4, tak krok zmenšujeme. Pokud se přijetí drží v rozmezí <0.4,0.6>, tak je pro nás krok optimální  nemusíme dělat nic.

In [110]:
def metropolis_criterion(delta: float, temperature: float) -> float:
    """
    Calculate probabilty if accept the candidate.

    Args:
        delta (float): difference between the candidate and current position
        temperature (float): Parameter which influence probability f the candidate is worse.Bigger value means bigger result probability.

    Returns:
        float: probability between 0 ... 1
    """
    if delta > 0:
        return np.exp(-delta / temperature).item()
    return 1


def corana_update(
    step_size: np.ndarray,
    accepted: np.ndarray,
    step_epoch: int,
    step_scaling: float = 2,
) -> np.ndarray:
    """
    Update the dynamic step size on end of the epoch.
    If the acceptance is smaller then 40%, make bigger step.Else if the step acceptance is bigger then 60%, reduce h step size.
    Otherwise don§t update the step.

    Args:
        step_size (np.ndarray): current step size
        accepted (np.ndarray): How many candidates were accepted in current epoch
        step_epochs (int): How many candidates are generated for every epoch.
        step_scaling (float, optional): Parameter settings, which control the variability in stepsize. Defaults to 2.

    Returns:
        np.ndarray: Upadated pstep_size
    """
    for idx in range(step_size.shape[0]):
        acceptence = accepted[idx] / step_epoch
        if acceptence < 0.4:
            step_size[idx] /= 1 + step_scaling * (0.4 - acceptence) / 0.4
        elif acceptence > 0.6:
            step_size[idx] *= 1 + step_scaling * (acceptence - 0.6) / 0.4
    return step_size


def simmulated_annealing(
    point: npt.NDArray,
    step_size: npt.NDArray,
    function: Callable = scipy.optimize.rosen,
    epochs: float = 20,
    temperature: float = 10.0,
    temperature_decay: float = 0.8,
    steps_epoch: int = 1000,
    step_scaling: float = 1.0,
) -> tuple[float, int, int, pd.DataFrame]:
    """
    Simmulated Annealing. Optimization algorithm which finds minimas. The function stop after n epochs without new global minima.
    Needs to be calibrated and run several times to make sure of the result.

    Args:
        point (npt.NDArray): Initial point in 2D
        step_size (npt.NDArray): Initial dynamic stepsize defined in each dimension.
        function (Callable, optional): Function for minima finding. Defaults to scipy.optimize.rosen.
        epochs (float, optional): Stopping criteria. Maximal count of cycles without new minima. Defaults to 20.
        temperature (float, optional): Variable which affects probability of accepting wrse candidate. Defaults to 10..
        temperature_decay (float, optional): Multiplication constants which is applied on end of every epoch. Defaults to 0.8.
        steps_epoch (int, optional): How many trials to generate in epoch. Defaults to 1000.
        step_scaling (float, optional): Scaling variables which affects dynamic step size scaling. Defaults to 1..

    Returns:
        tuple[float,int,int,pd.DataFrame]: Results of the algorithm with data for animation.
    """
    step_size = np.copy(step_size)
    cycles = 0
    eval = 0
    current_point = np.copy(point)
    best_value = function(current_point)
    bad_epoch = (
        -1
    )  # epoch counter when new best isn't found, zeroed when new best found
    rng = np.random.default_rng()
    points_frame_data = [[current_point[0], current_point[1], best_value, 0]]

    # infinite loop, the methods only ends when the tolerance criterion is met (all last n accepted candidates difference rom best)
    while True:
        cycles += 1
        accepted = np.zeros(
            point.shape[0]
        )  # counter of how many candidates were accepted in each dimension, need to be reset on every new epoch
        bad_epoch += 1

        # inner epoch after which the temeprature is scaled down a step size recalculated
        for _ in range(steps_epoch):
            # checking every dimension separately
            for idx in range(accepted.shape[0]):
                new_point = current_point.copy()
                new_point[idx] = (
                    current_point[idx] + rng.uniform(-1, 1) * step_size[idx]
                )  # calculate new candidate with dynamic step size
                new_value = function(new_point)

                # append the acepted candidate to table for later use in animation
                points_frame_data.append(
                    [new_point[0], new_point[1], new_value, cycles]
                )

                eval += 1
                # calculate and check probability if accept the generated candidate
                if rng.uniform(0, 1) < metropolis_criterion(
                    new_value - best_value, temperature
                ):
                    current_point[idx] = new_point[idx]
                    accepted[
                        idx
                    ] += 1  # raise counter of acceptance in current dimension
                    if new_value < best_value:
                        best_value = new_value
                        bad_epoch = 0

        # check the criterion difference against the latest point
        if bad_epoch > epochs:
            return (
                best_value,
                cycles,
                eval,
                pd.DataFrame(
                    points_frame_data,
                    columns=["x_coord", "y_coord", "value", "iteration"],
                ),
            )
        # downscale the temperature on for next cycle
        temperature *= temperature_decay

        # update the dynamic step size
        step_size = corana_update(
            step_size=step_size,
            accepted=accepted,
            step_epoch=steps_epoch,
            step_scaling=step_scaling,
        )

In [115]:
res = measure(
    simmulated_annealing,
    trials=10,
    point=np.asarray([0.0, -5.0]),
    step_size=np.asarray([2.0, 2.0]),
    step_scaling=1,
    temperature=100,
    temperature_decay=0.9,
    epochs=10,
    steps_epoch=100,
    function=himmel_baum,
)

avg. time: 0.05043366629979573 +-0.022713408555228732 seconds, result: 0.038007808234579527 +-0.08656786629454763, iterations: 24 +-9.140872800534725, evaluations: 4800 +-1828.1745601069451


In [ ]:
animate(
    res,
    frame_duration=800,
    function=himmel_baum,
    x_min=-5,
    x_max=5,
    y_min=-5,
    y_max=5,
    mesh_points_x=500,
    mesh_points_y=500,
    marker_size=2,
).show()

## algoritmus světlušky

Tento algoritmus je inspirován chováním světlušek, kdy samičky vábí amce za pomocí světla. V tomto algoritmu každý jedinec vábí ostatní  horším řešením k sobě. Světluška a jde směrem světlušce b s lepším řešením podle následujících rovnic.
$$
a = a + \beta I(||b - a||)(b - a) + \alpha e
\\
I(r)= e^{-\gamma r^2}
$$

Tyto rovnice mají 3 hyperparametry. $\alpha$ určuje velikost náhodné složky, $\beta$ určuje velikost složky intenzity světla. $\gamma$ pak ovlivňuje samotné pohlcování světla.

In [20]:
rng = np.random.default_rng()


def brightness(
    distance: float,
    gamma: float = 1.0,
) -> float:
    return np.exp(-gamma * distance).item()


@dataclass
class Firefly:
    x: float
    y: float
    z: float = float("inf")

    def dist(self, other: Self):
        """
        Euclidean distance.

        Args:
            other (Self): Other firefly to which calculate distance.
        """
        return np.sqrt((other.x - self.x) ** 2 + (other.y - self.y) ** 2).item()

    def new_position(
        self, other: Self, beta: float = 1.0, gamma: float = 1.0, alpha: float = 1
    ):
        """
        Calculate new position for this firefly against the other firely.

        Args:
            other (Self): Other firefly to which get closer.
            beta (float, optional): Intensity of the source light. Defaults to 1
            gamma (float, optional): Light absorption coefficient. Defaults to 1.
        """
        calculated_brightness = beta * brightness(self.dist(other), gamma)
        self.x += calculated_brightness * (other.x - self.x) + alpha * rng.normal()
        self.y += calculated_brightness * (other.y - self.y) + alpha * rng.normal()


def firefly(
    population_size: int,
    function: Callable = scipy.optimize.rosen,
    epochs_max: int = 1000,
    alpha: float = 1.0,
    beta: float = 1.0,
    gamma=1,
    init_min: int = -1,
    init_max: int = 1,
) -> tuple[float, int, int, pd.DataFrame]:
    """
    Firefly algorithm. There's not any stopping criterya, the algorithm will stop after predefined n epochs.

    Args:
        population_size (int): Variable which controls how many points tests and update every epoch.
        function (Callable, optional): Minimalization function .Defaults to scipy.optimize.rosen.
        epochs_max (int, optional): Maximal it. Defaults to 1000.
        beta (float, optional): Variable which gives weight to lightning source of firefly. Defaults to 1.
        alpha  float, pional): Variable which defines size of random walk.
        gamma (int, optional): Light absorption variable which affects intensity calculation. Defaults to 1.
        init_min (int, optional): Initial minimal random value for each axis. Defaults to -1.
        init_max (int, optional): Initial maximal random value for each axis. Defaults to 1.

    Returns:
        tuple[float, int, int, pd.DataFrame]: Algorithm results with animation data.
    """

    dim = 2
    eval = 0
    iteration = 0
    positions = []

    # Initialize fireflies with random position
    fireflies = [
        Firefly(*rng.uniform(init_min, init_max, dim).tolist())
        for _ in range(population_size)
    ]

    # init initial evaluations
    for firefly in fireflies:
        firefly.z = function(np.asarray((firefly.x, firefly.y)))
        positions.append((firefly.x, firefly.y, firefly.z, iteration))

    for t in range(epochs_max):
        iteration += 1

        # calculate new position for each firefly
        for current_firefly in fireflies:
            for checking_firefly in fireflies:
                if current_firefly is checking_firefly:
                    continue
                if current_firefly.z > checking_firefly.z:
                    current_firefly.new_position(checking_firefly, beta, gamma, alpha)

        # evaluate new position for each firefly
        for firefly in fireflies:
            firefly.z = function(np.asarray((firefly.x, firefly.y)))

            # record fireflies to table
            positions.append((firefly.x, firefly.y, firefly.z, iteration))

        eval += population_size

    # get best firefly (lowest z)
    best_value = fireflies[0].z
    for firefly in fireflies[1:]:
        best_value = firefly.z if firefly.z < best_value else best_value

    return (
        best_value,
        iteration,
        eval,
        pd.DataFrame(positions, columns=["x_coord", "y_coord", "value", "iteration"]),
    )

In [26]:
res = measure(
    firefly,
    trials=10,
    alpha=0.1,
    function=himmel_baum,
    population_size=100,
    gamma=0.0002,
    beta=0.005,
    epochs_max=30,
    init_min=-5,
    init_max=5,
)

avg. time: 0.8430424714999389 +-0.14461158696777543 seconds, result: 0.015029476513893683 +-0.011388818269248036, iterations: 30 +-0.0, evaluations: 3000 +-0.0


In [ ]:
animate(
    res,
    frame_duration=100,
    function=himmel_baum,
    x_min=-5,
    x_max=5,
    y_min=-5,
    y_max=5,
    mesh_points_x=500,
    mesh_points_y=500,
    marker_size=2,
).show()

## zdroje



KOCHENDERFER, Mykel a Tim WHEELER. Algorithms for Optimization [online]. Massachusetts: MIT Press, 2019 [cit. 2023-09-25]. ISBN 978-0262039420. Dostupné z: https://algorithmsbook.com/optimization/files/optimization.pdf